In [1]:
import numpy as np
import pandas as pd 
import datetime as datetime
import gym
from gym import envs

from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import Dense, Embedding, Reshape
from tensorflow.keras.optimizers import Adam    

import os
import time

import random 
from collections import deque

envs.registry.all()
env = gym.make('CartPole-v0')

import wandb


from wandb.keras import WandbCallback



C:\ProgramData\Anaconda3\lib\site-packages\ale_py\roms\__init__.py:94: DeprecationWarning: Automatic importing of atari-py roms won't be supported in future releases of ale-py. Please migrate over to using `ale-import-roms` OR an ALE-supported ROM package. To make this warning disappear you can run `ale-import-roms --import-from-pkg atari_py.atari_roms`.For more information see: https://github.com/mgbellemare/Arcade-Learning-Environment#rom-management
  _RESOLVED_ROMS = _resolve_roms()


In [2]:
class Agent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        
        #experience replay
        self.memory = deque(maxlen = 2000)
        
        #discount rate
        self.gamma = 0.95
        
        #epsilon-greedy params
        self.epsilon = 1.0
        self.epsilon_decay = 0.995
        self.epsilon_min = 0.0001
        
        self.learning_rate = 0.001
        
        self.model = self._build_model()
        self.target = self._build_model()
        self.alighn_target_model()
    
    def _build_model(self):
        
        wandb.init(config = {
            "gamma" : 0.95,
            "epsilon_decay" : 0.995,
            "learning_rate": 0.0001,
            "epochs": 1,
            "batch_size": 30,
            "neuroneLayer1" : 30,
            "neuroneLayer2" : 24,
            "activation1" : "relu",
            "activation2" : "relu",
            "activation_out" : "linear"
        })
        
        
        model = Sequential()
        
        #hyper params to tune
        model.add(Dense(30, input_dim = self.state_size, activation = 'relu'))
        model.add(Dense(24, activation = 'relu'))
        model.add(Dense(self.action_size, activation = 'linear'))
        
        model.compile(loss = 'mse', optimizer = Adam(lr = self.learning_rate))
        
        return model
    
    def alighn_target_model(self):
        self.target.set_weights(self.model.get_weights())
    
    def remember(self, state, action,reward,next_state,done):
        self.memory.append((state, action, reward, next_state, done))
        
    def act(self, state):
        #epsilon-greedy choice of the action to perform
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])
    
    def replay(self, batch_size):  
    
        x = np.ndarray((0,4))
        y = np.ndarray((0,2))
        
        
        minibatch = random.sample(self.memory, batch_size)
        
        for state, action, reward, next_state, terminated in minibatch:
            
            target = self.model.predict(state)
            
            if terminated:
                target[0][action] = reward
            else:
                t = self.target.predict(next_state)
                target[0][action] = reward + self.gamma * np.amax(t)
            
            #x = np.vstack((x, state))
            #y = np.vstack((y, target_f))
        
        
        
            self.model.fit(state, target, epochs =1, verbose = 0,
            callbacks=[WandbCallback()])
        
        
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
        
    def load(self, path):
        self.model.load_weights(path)
        
    def save(self, path):
        self.model.save_weights(path)

In [ ]:
#sort of main, to define as a class "controller"

!python -m wandb login 46004e4ab31134349e71ceede20423c7cfdbb092
wandb.init(project="RL_PLAY", entity="devantheryl")



env = gym.make('CartPole-v0')
state_size = env.observation_space.shape[0]
action_size = env.action_space.n
batch_size = 30
n_episode = 1000
output_dir = 'model_output/cartepole'

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

agent = Agent(state_size, action_size)

for e in range(n_episode):
    
    state = env.reset()
    state = np.reshape(state, [1,state_size])
    done = False
    score = 0
    
    while not done:
        env.render()
        score+=1
        action = agent.act(state)
        
        next_state, reward, done, _ = env.step(action)
        next_state = np.reshape(next_state, [1,state_size])
        
        agent.remember(state, action, reward, next_state, done)
        
        state = next_state
        
        if done:
            print("episode : {}/{}, score : {}, e : {:.2}".format(e, n_episode, score, agent.epsilon))
            agent.alighn_target_model()
            break
    
    
    if len(agent.memory) > batch_size:
            
            
            agent.replay(batch_size)
            

    if e%50 == 0:
        #agent.save(output_dir + "weights_" + '{:04d}'.format(e) + ".hdf5")
        pass

env.close()
    
        

wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\LDE/.netrc
wandb: Currently logged in as: devantheryl (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.7 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


wandb: wandb version 0.12.7 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


C:\ProgramData\Anaconda3\lib\site-packages\keras\optimizer_v2\optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


wandb: wandb version 0.12.7 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


episode : 0/1000, score : 73, e : 1.0
episode : 1/1000, score : 16, e : 0.99
episode : 2/1000, score : 12, e : 0.99
episode : 3/1000, score : 16, e : 0.99
episode : 4/1000, score : 16, e : 0.98
episode : 5/1000, score : 15, e : 0.98
episode : 6/1000, score : 19, e : 0.97
episode : 7/1000, score : 20, e : 0.97
episode : 8/1000, score : 15, e : 0.96
episode : 9/1000, score : 35, e : 0.96
episode : 10/1000, score : 11, e : 0.95
episode : 11/1000, score : 18, e : 0.95
episode : 12/1000, score : 12, e : 0.94
episode : 13/1000, score : 10, e : 0.94
episode : 14/1000, score : 23, e : 0.93
episode : 15/1000, score : 37, e : 0.93
episode : 16/1000, score : 34, e : 0.92
episode : 17/1000, score : 51, e : 0.92
episode : 18/1000, score : 21, e : 0.91
episode : 19/1000, score : 16, e : 0.91
episode : 20/1000, score : 21, e : 0.9
episode : 21/1000, score : 20, e : 0.9
episode : 22/1000, score : 36, e : 0.9
episode : 23/1000, score : 13, e : 0.89
episode : 24/1000, score : 18, e : 0.89
episode : 25/1

episode : 204/1000, score : 36, e : 0.36
episode : 205/1000, score : 84, e : 0.36
episode : 206/1000, score : 51, e : 0.36
episode : 207/1000, score : 48, e : 0.35
episode : 208/1000, score : 43, e : 0.35
episode : 209/1000, score : 46, e : 0.35
episode : 210/1000, score : 49, e : 0.35
episode : 211/1000, score : 40, e : 0.35
episode : 212/1000, score : 115, e : 0.35
episode : 213/1000, score : 53, e : 0.34
episode : 214/1000, score : 75, e : 0.34
episode : 215/1000, score : 30, e : 0.34
episode : 216/1000, score : 52, e : 0.34
episode : 217/1000, score : 42, e : 0.34
episode : 218/1000, score : 123, e : 0.34
episode : 219/1000, score : 42, e : 0.33
episode : 220/1000, score : 23, e : 0.33
episode : 221/1000, score : 40, e : 0.33
episode : 222/1000, score : 47, e : 0.33
episode : 223/1000, score : 98, e : 0.33
episode : 224/1000, score : 45, e : 0.33
episode : 225/1000, score : 75, e : 0.32
episode : 226/1000, score : 95, e : 0.32
episode : 227/1000, score : 59, e : 0.32
episode : 228/

episode : 403/1000, score : 200, e : 0.13
episode : 404/1000, score : 190, e : 0.13
episode : 405/1000, score : 200, e : 0.13
episode : 406/1000, score : 200, e : 0.13
episode : 407/1000, score : 200, e : 0.13
episode : 408/1000, score : 200, e : 0.13
episode : 409/1000, score : 84, e : 0.13
episode : 410/1000, score : 200, e : 0.13
episode : 411/1000, score : 72, e : 0.13
episode : 412/1000, score : 118, e : 0.13
episode : 413/1000, score : 200, e : 0.13
episode : 414/1000, score : 132, e : 0.13
episode : 415/1000, score : 200, e : 0.12
episode : 416/1000, score : 126, e : 0.12
episode : 417/1000, score : 28, e : 0.12
episode : 418/1000, score : 105, e : 0.12
episode : 419/1000, score : 200, e : 0.12
episode : 420/1000, score : 200, e : 0.12
episode : 421/1000, score : 200, e : 0.12
episode : 422/1000, score : 200, e : 0.12
episode : 423/1000, score : 200, e : 0.12
episode : 424/1000, score : 128, e : 0.12
episode : 425/1000, score : 61, e : 0.12
episode : 426/1000, score : 133, e : 0

episode : 597/1000, score : 200, e : 0.05
episode : 598/1000, score : 127, e : 0.05
episode : 599/1000, score : 200, e : 0.05
episode : 600/1000, score : 200, e : 0.049
episode : 601/1000, score : 200, e : 0.049
episode : 602/1000, score : 200, e : 0.049
episode : 603/1000, score : 156, e : 0.049
episode : 604/1000, score : 200, e : 0.048
episode : 605/1000, score : 200, e : 0.048
episode : 606/1000, score : 23, e : 0.048
episode : 607/1000, score : 198, e : 0.048
episode : 608/1000, score : 94, e : 0.047
episode : 609/1000, score : 191, e : 0.047
episode : 610/1000, score : 200, e : 0.047
episode : 611/1000, score : 165, e : 0.047
episode : 612/1000, score : 143, e : 0.047
episode : 613/1000, score : 150, e : 0.046
episode : 614/1000, score : 200, e : 0.046
episode : 615/1000, score : 187, e : 0.046
episode : 616/1000, score : 144, e : 0.046
episode : 617/1000, score : 56, e : 0.045
episode : 618/1000, score : 95, e : 0.045
episode : 619/1000, score : 87, e : 0.045
episode : 620/1000,

In [ ]:
state = np.array([[1,1,1,1]])

x = np.ndarray((0,4))

x = np.vstack((x, state))
x = np.vstack((x, state))
x = np.vstack((x, state))
x = np.vstack((x, state))



print(x)